<a href="https://colab.research.google.com/github/vivekvijay107/Ai_image_enhancment-using-pretrained-model_SCI-/blob/main/sci_image_enhnc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok Pillow numpy matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.1 MB/s eta 0:00:00


In [2]:
!ngrok authtoken 2zVdvqhZORegABQjSwRTOist41T_32ePwsqssb78LrapVDADe


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!git clone https://github.com/vis-opt-group/SCI.git
%cd SCI

!mkdir -p weights
!wget https://github.com/vis-opt-group/SCI/raw/main/weights/medium.pt -O weights/medium.pt


Cloning into 'SCI'...
remote: Enumerating objects: 414, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 414 (delta 145), reused 174 (delta 126), pack-reused 211 (from 1)
Receiving objects: 100% (414/414), 45.15 MiB | 34.95 MiB/s, done.
Resolving deltas: 100% (174/174), done.
/content/SCI
--2025-07-07 19:50:16--  https://github.com/vis-opt-group/SCI/raw/main/weights/medium.pt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vis-opt-group/SCI/main/weights/medium.pt [following]
--2025-07-07 19:50:16--  https://raw.githubusercontent.com/vis-opt-group/SCI/main/weights/medium.pt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.c

In [4]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
import numpy as np
from torchvision import transforms
from torchvision.transforms import ToPILImage
from model import Finetunemodel

# ✅ Patch Finetunemodel to load weights on CPU
def patched_init(self, weights, map_location='cpu'):
    super(Finetunemodel, self).__init__()
    from model import EnhanceNetwork, LossFunction
    self.enhance = EnhanceNetwork(layers=1, channels=3)
    self._criterion = LossFunction()

    base_weights = torch.load(weights, map_location=map_location)
    model_dict = self.state_dict()
    pretrained_dict = {k: v for k, v in base_weights.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    self.load_state_dict(model_dict)

Finetunemodel.__init__ = patched_init

@st.cache_resource
def load_model():
    model = Finetunemodel('weights/medium.pt', map_location='cpu')
    model.eval()
    return model

model = load_model()

# Streamlit UI
st.set_page_config(page_title="SCI Image Enhancer", layout="centered")
st.title("🖼️ SCI Image Enhancer")
st.write("Upload a low-light or low-quality image to enhance it using the **SCI AI model**.")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Original Image", use_column_width=True)

    transform = transforms.Compose([transforms.ToTensor()])
    input_tensor = transform(image).unsqueeze(0)

    with torch.no_grad():
        _, enhanced = model(input_tensor)
        enhanced = torch.clamp(enhanced, 0, 1)[0]
        enhanced_image = ToPILImage()(enhanced)

    st.image(enhanced_image, caption="✨ Enhanced Image", use_column_width=True)

    # Optional download
    from io import BytesIO
    buf = BytesIO()
    enhanced_image.save(buf, format="JPEG")
    byte_im = buf.getvalue()
    st.download_button("📥 Download Enhanced Image", byte_im, "enhanced.jpg", "image/jpeg")


Writing app.py


In [ ]:
from pyngrok import ngrok

# Kill if already running
!kill $(pgrep streamlit) > /dev/null 2>&1

# Open public URL
public_url = ngrok.connect(8501)
print("🔗 Open your app here:", public_url)

# Run Streamlit app
!streamlit run app.py &


🔗 Open your app here: NgrokTunnel: "https://1c6c3f8babc7.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.219.15:8501

